# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [20]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# Q1,Q2

Two sample z-test Null Hypotheses can be applied.
H0                   : proportion (black) = proportion (white) 
Alternate Hypotheses : proportion (black) != proportion (white)

CLT can be applied since the data size is adequate and 'b' and 'w' values are random.


In [8]:
df2 = data.groupby(['race','call']).count()
df2.head()

id    ad  education  ofjobs  yearsexp  honors  volunteer  \
race call                                                               
b    0.0   2278  2278       2278    2278      2278    2278       2278   
     1.0    157   157        157     157       157     157        157   
w    0.0   2200  2200       2200    2200      2200    2200       2200   
     1.0    235   235        235     235       235     235        235   

           military  empholes  occupspecific    ...      compreq  orgreq  \
race call                                       ...                        
b    0.0       2278      2278           2278    ...         2278    2278   
     1.0        157       157            157    ...          157     157   
w    0.0       2200      2200           2200    ...         2200    2200   
     1.0        235       235            235    ...          235     235   

           manuf  transcom  bankreal  trade  busservice  othservice  missind  \
race call                                                                      
b    0.0    2278      2278      2278   2278        2278        2278     2278   
     1.0     157       157       157    157         157         157      157   
w    0.0    2200      2200      2200   2200        2200        2200     2200   
     1.0     235       235       235    235         235         235      235   

           ownership  
race call             
b    0.0        2278  
     1.0         157  
w    0.0        2200  
     1.0         235  

[4 rows x 63 columns]

Totally 2435 resumes.  Black received 157, white received 235 calls.

# Q3

In [37]:
bcall_mean=data[data.race=='b'].call.mean()
wcall_mean=data[data.race=='w'].call.mean()
print ('Black mean: ' +str(bcall_mean))
print ('White mean: ' +str(wcall_mean))
diff_mean = wcall_mean - bcall_mean
print ('Difference : ' + str(diff_mean))

Black mean: 0.0644763857126236
White mean: 0.09650924056768417
Difference : 0.03203285485506058


In [39]:
w_call = np.sum(data[data.race=='w'].call)
b_call = np.sum(data[data.race=='b'].call)
w_len = len(data[data.race=='w'])
b_len = len(data[data.race=='b'])            
phat = (w_call + b_call) / (w_len + b_len)
z = (diff_mean) / np.sqrt(phat * (1 - phat) * (( 1/ float(w_len)) + (1/float(b_len))))
print ('z score = ' + str(z))
p = stats.norm.cdf(-z)*2
print ('p-value = ' + str(p))

z score = 4.108412235238472
p-value = 3.983885409504418e-05


In [42]:
margin_e = 1.96 * np.sqrt(phat * (1 - phat) * (( 1/ float(w_len)) + (1/float(b_len))))
conf = diff_mean + np.array([-1, 1]) * margin_e
print('Margin of Error = '+ str(margin_e))
print('Confidence interval = '+ str(conf))

Margin of Error = 0.015281912310894095
Confidence interval = [0.01675094 0.04731477]


# Q4,Q5


Null hypothesis should be rejected because p-value is so close to 0. Call back rate is not the same as expected. Although there is a significant different in call back rates, race might not be the number one reason. Other parameters should also be checked.

In [43]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.44902858418937
p-value: 4.997578389963255e-05
